In [1]:
!pip install -q pyiceberg

In [3]:
from pyiceberg.catalog.rest import RestCatalog
import logging

# logging.basicConfig(level=logging.DEBUG)

import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa

CATALOG_URL = "http://lakekeeper:8181/catalog"
DEMO_WAREHOUSE = "irisa-ot"

catalog = RestCatalog(
    name="banking_catalog",
    warehouse=DEMO_WAREHOUSE,
    uri=CATALOG_URL,
    token="dummy",
)


In [4]:
# Create a new namespace if it doesn't already exist
test_namespace = ("banking",)
if test_namespace not in catalog.list_namespaces():
    catalog.create_namespace(test_namespace)


In [4]:
# Write data
test_table = ("banking", "source_transactions")
df = pd.DataFrame(
    {
        "id": [1, 2, 3],
        "data": ["a", "b", "c"],
    }
)
pa_df = pa.Table.from_pandas(df)

if test_table in catalog.list_tables(namespace=test_namespace):
    catalog.drop_table(test_table)

table = catalog.create_table(
    test_table,
    schema=pa_df.schema,
)

table.append(pa_df)

In [6]:
test_table = ("banking", "source_transactions")
# Read data
table = catalog.load_table(test_table)
table.scan().to_pandas()

,transaction_id,source_system,transaction_date,amount,account_id,transaction_type,reference_id,status,payload,created_at,processing_timestamp
0,CB-851C68DDEA69,core_banking,2025-07-12 01:56:51+00:00,856.05,ACC37286176,transfer,REF-2413692350,reversed,"""{""""description"""": """"Here out off management g...",NaT,NaT
1,CB-2E469C1D69BE,core_banking,2025-07-12 20:48:47+00:00,7021.10,ACC61141394,fee,REF-9737058377,reversed,"""{""""description"""": """"Several question fall.""""",NaT,NaT
2,CB-8B543EEB2254,core_banking,2025-07-12 22:15:50+00:00,7568.74,ACC48148385,withdrawal,REF-2435528379,failed,"""{""""description"""": """"Task president program ma...",NaT,NaT
3,CB-8EE987C8F7E3,core_banking,2025-07-12 17:37:26+00:00,5137.10,ACC98329963,refund,REF-9353449336,failed,"""{""""description"""": """"Open thus kitchen keep re...",NaT,NaT
4,CB-479AB84B2591,core_banking,2025-07-12 00:39:33+00:00,3477.77,ACC97584503,deposit,REF-7565398291,pending,"""{""""description"""": """"Soon area history fall pl...",NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...
4995,CB-4B931057A6BD,core_banking,2025-06-27 00:24:54+00:00,9211.29,ACC45298962,deposit,REF-6365246688,pending,"""{""""description"""": """"Produce certain according...",NaT,NaT
4996,CB-C7D61D6A1C0C,core_banking,2025-06-27 06:39:22+00:00,8471.04,ACC24923570,payment,REF-5994880482,reversed,"""{""""description"""": """"Strong Mrs foot why feel ...",NaT,NaT
4997,CB-C75AE7049776,core_banking,2025-06-27 10:23:48+00:00,1951.25,ACC50743151,payment,REF-7116344828,pending,"""{""""description"""": """"Fund bag impact degree go...",NaT,NaT
4998,CB-A32E30574E25,core_banking,2025-06-27 18:05:15+00:00,1478.65,ACC49581697,withdrawal,REF-4870509018,failed,"""{""""description"""": """"Pm crime anyone enter rad...",NaT,NaT
